## results:
Loss: 0.6226252913475037 
Acc:0.7956989407539368 
F1 score: 0.8069892525672913
Precision: 0.6920763850212097
Recall: 0.9816280007362366

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load the dataset
from pandas import read_csv
from scipy.special import expit
from sklearn.utils import shuffle
import pickle
def load_dataset_train(full_path = '/content/drive/MyDrive/Privacy/real_data_train.csv'):
  # load the dataset as a numpy array
  with open(full_path, 'rb') as f:
    data = pickle.load(f)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X_train, y_train = load_dataset_train()
X_train = expit(X_train)
print(X_train.shape, y_train.shape)
print(X_train[0])


def load_dataset_test(full_path = '/content/drive/MyDrive/Privacy/real_data_test.csv'):
  # load the dataset as a numpy array
  with open(full_path, 'rb') as f:
    data = pickle.load(f)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X_test, y_test = load_dataset_test()
X_test = expit(X_test)
print(X_test.shape)

(1000, 4) (1000,)
[0.99411941 0.99982842 0.05258531 0.18114637]
(372, 4)


In [ ]:
from pandas import read_csv
dataframe = read_csv("/content/drive/MyDrive/Privacy/DPCGAN_data_new_1000.csv", header=None)
data = dataframe.values[:,1:]
print(data[2])

dataframe = read_csv('/content/drive/MyDrive/Privacy/DPCGAN_labels_new_1000.csv', header=None)
labels = dataframe.values[:,1]
print(labels.shape)

X_DPCGAN = data
Y_DPCGAN = labels

print(X_DPCGAN.shape)
print(Y_DPCGAN.shape)

[0.4099354  0.75167114 0.8109339  0.1232217 ]
(1001,)
(1001, 4)
(1001,)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
import tensorflow as tf


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


model = Sequential([
Dense(input_dim = 4, units = 2, activation = 'relu'),
# Dense(units = 32, activation = 'relu'),
# Dropout(0.2),
Dense(units = 10, activation = 'relu'),
Dense(units = 4, activation = 'relu'),
Dense(units =1, activation = 'sigmoid'),])


In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', f1_m, precision_m, recall_m])
model.fit(X_DPCGAN, Y_DPCGAN, batch_size = 15, epochs = 15)
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print(loss, accuracy, f1_score, precision, recall)

Epoch 1/15
67/67 [==============================] - 1s 1ms/step - loss: 0.6955 - accuracy: 0.4547 - f1_m: 0.6067 - precision_m: 0.4716 - recall_m: 0.8972
Epoch 2/15
67/67 [==============================] - 0s 1ms/step - loss: 0.6931 - accuracy: 0.4889 - f1_m: 0.2752 - precision_m: 0.3692 - recall_m: 0.2930
Epoch 3/15
67/67 [==============================] - 0s 1ms/step - loss: 0.6927 - accuracy: 0.5329 - f1_m: 0.1183 - precision_m: 0.3591 - recall_m: 0.0739
Epoch 4/15
67/67 [==============================] - 0s 2ms/step - loss: 0.6906 - accuracy: 0.5803 - f1_m: 0.3246 - precision_m: 0.7119 - recall_m: 0.2730
Epoch 5/15
67/67 [==============================] - 0s 1ms/step - loss: 0.6868 - accuracy: 0.5346 - f1_m: 0.4960 - precision_m: 0.5636 - recall_m: 0.5559
Epoch 6/15
67/67 [==============================] - 0s 1ms/step - loss: 0.6844 - accuracy: 0.5357 - f1_m: 0.5539 - precision_m: 0.5584 - recall_m: 0.6451
Epoch 7/15
67/67 [==============================] - 0s 2ms/step - loss: 0.68